In [ ]:
#
# Compressibility match and forecast
# v1.0, D Robbins, 17/01/2020
#
# -> Code workbook to perform RMU compressibility matching to drive impact of IPC/IIC estimates
#    on forecast reservoir pressures
# -> Side effect is also to update the cumulative offtake calculator on the PRES data plots
#
# Global functions and libraries
from pyspark.sql import functions as F
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
#
# Global parameters
f_bo = 1.16     # oil FVF (rb/stb)
f_bw = 1.006    # water FVF (rb/stb)
f_bg = 0.836    # gas FVF (rb/mscf)
#
dt_forecast_start = '2020-02-01' # forecast start date
#
# list of RMUS on field (example)
rmu_list = ['RMU1', 'RMU2']
#
# solution GOR, aquifer size, compressibility dictionary
# -> this is used to override the values in RMU_compressibility input IF requested 
#    (i.e. use this for "history matching" with i_use_global_compressibility = TRUE in transform historical_pressures)
i_RMUMBAL_GOR = 0               # right now, this doesn't do anything
i_RMUMBAL_aquifer = 1           # Wi small pot model, aquifer productivity (rb/d/psi)
i_RMUMBAL_aquifer_delay = 2     # number of days of delay for aquifer action
i_RMUMBAL_compressibility = 3   # psi/mmbbl, large/gassy tanks could be >200, small/watery tanks <100
i_RMUMBAL_pinit = 4             # initial pressure at datum (psia)
i_RMUMBAL_dinit = 5             # initial date that initial pressure is reported to (not yet used, for the future)
#
# example dictionary
d_RMU_MBAL = {
    'RMU1': [342.0, 5.0, 50, 450.0, 3005.0, '2017-01-01'],
    }
#
# WELLS-PER-RMU DICTIONARY (dict(key)[i_rmuwell_dict_*][list of n_wells/limits)
#
i_dict_rmuwell_wells = 0	# list of wells in RMU
#
# don't need the limits, just the well mapping
# example mapping
Q204_RMU_dictionary = {
    				   'RMU1':[['P01','P02','I01','I02']],
					   }
#

In [ ]:
def RMU_rollup_historical (Rolled_well_data, Forecasting_inputs_forecast_allocation_factors, Data_pull):
    #
    # Transform to roll up well cumulatives into a cumulative RMU offtake
    #   based on the allocation factors from the forecasting inputs
    #
    # NOTE: this is slightly different to the cumulative offtake calculator, and is
    #   intended to test different match scenarios for RMU MBAL
    #
    # take copy of dfs
    prod_alloc_df = Rolled_well_data.copy(deep=True)
    allocf_df = Forecasting_inputs_forecast_allocation_factors.copy(deep=True)
    #
    # get list of timestamps
    df_prod_data = Data_pull.copy(deep=True).sort_values(by='datetime')
    time_list = df_prod_data['datetime'].tolist()
    #
    # set up final allocation factor dataframe by merging allocation df and production dataframe
    final_af_df = pd.merge(prod_alloc_df, allocf_df, on='well')
    final_af_df = final_af_df.fillna(0)
    #
    l_rmu = rmu_list
    # loop over RMUs and calculate cumulative offtake
    for rmu in l_rmu:
        #
        # set up RMU df
        rmu_df = pd.DataFrame(time_list, columns=['datetime'])
        #
        # insert RMU into df
        rmu_df.loc[:,'rmu'] = rmu
        #
        # add this to all RMU df
        try:
            all_rmu_df = pd.concat([all_rmu_df,rmu_df],sort=False)
        except:
            all_rmu_df = rmu_df.copy(deep=True)
        #
    #
    # set up offtake lists for rollup loop
    rmu_offtake_list = []
    rmu_cum_offtake_list = []
    #
    # loop over df and derive offtake
    for i1, row in all_rmu_df.iterrows():
        #
        rmu = row['rmu']
        #
        offtake = 0.0
        cum_offtake = 0.0
        #
        # get row of the final allocation df with same date time
        time_prod_df = final_af_df.loc[final_af_df['datetime'] == row['datetime']]
        #
        # loop over allocation df and sum offtake/cum offtake over wells at this point in time
        for i2, inner_row in time_prod_df.iterrows():
            #
            offtake += inner_row['offtake_rate'] * inner_row[rmu]
            cum_offtake += inner_row['cum_offtake'] * inner_row[rmu]
        #
        # append offtake to RMU offtake list
        rmu_offtake_list.append(offtake)
        rmu_cum_offtake_list.append(cum_offtake)
        #
    #
    # append lists to df
    all_rmu_df.loc[:,'offtake_rate'] = rmu_offtake_list
    all_rmu_df.loc[:,'cum_offtake'] = rmu_cum_offtake_list
    #
    # reset datetime column name to not screw up the PRES tracker (THIS IS ALL MY FAULT FROM CHANGING THE DATA EXTRACTOR BEHAVIOUR, I'M ETERNALLY SORRY BUT I STILL BLAME PALANTIR)
    all_rmu_df.columns = ['time','rmu','offtake_rate','cum_offtake']
    #
    # return rolled up df
    return all_rmu_df

In [ ]:
def historical_pressures(RMU_rollup_historical, Forecasting_inputs_RMU_compressibility):
    #
    # Code transform to generate estimated pressures based on offtake, compressibility etc. for each RMU
    #
    # OPTIONS
    my_list_of_RMUs = rmu_list
    i_use_global_compressibility = True # if true, pull GOR/aquifer/compressibility data from global dictionary d_RMU_MBAL, otherwise pull from input table
    #
    # data pull
    df_RMU_offtake = RMU_rollup_historical.copy(deep=True)
    if i_use_global_compressibility == True:
        # use the global dictionary
        d_RMU_match = d_RMU_MBAL
        #
    else:
        # use the input df
        df_RMU_MBAL = Forecasting_inputs_RMU_compressibility.copy(deep=True)
        #
        # sort the df to be in the same expected order as the global dictionary
        df_RMU_MBAL = df_RMU_MBAL[['RMU','Solution_GOR','Aquifer','Aquifer_delay','Compressibility','Initial_datum_pressure','Initial_datum_date']]
        df_RMU_MBAL.set_index('RMU',inplace=True,drop=True)
        #
        # convert df into dictionary (I don't know why Pandas doesn't do the obvious key:list from columns, so use dictionary logic instead)
        d_RMU_match = {df_RMU_MBAL.index.values[i]:df_RMU_MBAL.values.tolist()[i] for i in range(len(df_RMU_MBAL.index.values))}
    #
    # now go through each RMU and calculate predicted pressure at this point
    for RMU in my_list_of_RMUs:
        #
        # get the RMU rollup df for the cum voidage offtake
        df_RMU = df_RMU_offtake[df_RMU_offtake['rmu']==RMU].copy(deep=True).sort_values(by='time').reset_index(drop=True)
        #
        # also get the starting pressure, compressibility and aquifer strength
        f_rmu_pi = d_RMU_match[RMU][i_RMUMBAL_pinit]
        f_rmu_c  = d_RMU_match[RMU][i_RMUMBAL_compressibility]
        f_rmu_aq = d_RMU_match[RMU][i_RMUMBAL_aquifer]
        i_rmu_aq_delay = d_RMU_match[RMU][i_RMUMBAL_aquifer_delay]
        #
        # loop over the times and compute estimated pressures
        l_rmu_p = [f_rmu_pi]        # initialise list of predicted pressures
        l_rmu_cumvoidage = [0.0]    # initialise cumulative voidage
        l_rmu_cumaquifer = [0.0]    # initialise cumulative aquifer influx
        #
        for i,row in df_RMU.iterrows():
            #
            if i > 0:
                #
                # get the voidage offtake (instantaneous)
                my_offtake = row['offtake_rate']*1e-6 # note offtake rates are in mmrbpd
                #
                # get aquifer influx (q_in (rb/d) = PI*dP, assume paq ~ pinit)
                if i > i_rmu_aq_delay:
                    my_aquifer = f_rmu_aq * (f_rmu_pi - l_rmu_p[i - i_rmu_aq_delay - 1]) # lower pres = positive aquifer influx
                else:
                    my_aquifer = 0.0
                #
                # convert aquifer influx into mmbbl/d
                my_aquifer *= 1e-6 # note offtake rates are in mmrbpd
                #
                # update pressure estimate (note convention: negative = offtake, positive = intake)
                #                             |RMU c|    | offtake|   |  small pot aquifer contribution   |
                l_rmu_p.append(l_rmu_p[i-1] + f_rmu_c * (my_offtake + my_aquifer))
                #
                # append cum voidage values (for plotting)
                l_rmu_cumvoidage.append(l_rmu_cumvoidage[i-1] + my_offtake)
                l_rmu_cumaquifer.append(l_rmu_cumaquifer[i-1] + my_aquifer)
            #
        #
        # add the computed pressure data to df and combine
        df_RMU.loc[:,'estimated_PRES'] = l_rmu_p
        df_RMU.loc[:,'cumulative_voidage'] = l_rmu_cumvoidage
        df_RMU.loc[:,'cumulative_aquifer'] = l_rmu_cumaquifer
        #
        try:
            df_RMU_pressures = pd.concat([df_RMU_pressures,df_RMU[['time','rmu','estimated_PRES','cumulative_voidage','cumulative_aquifer']]],ignore_index=True)
        except:
            df_RMU_pressures = df_RMU[['time','rmu','estimated_PRES','cumulative_voidage','cumulative_aquifer']]
    #
    return df_RMU_pressures

In [ ]:
def forecast_RMU_rollup(Forecasting_inputs_producer_forecast_IPC, Forecasting_inputs_forecast_IIC):
    #
    # Code transform to roll up estimated net offtake from each RMU
    #
    # take copies of IPC/IIC dfs
    df_IIC = Forecasting_inputs_forecast_IIC.copy(deep=True)
    df_IPC = Forecasting_inputs_producer_forecast_IPC.copy(deep=True)
    #
    l_df_well_data = []
    #
    # firstly, generate a rolled well data df for producers
    for well in df_IPC['Well'].unique():
        #
        # slice df and get information
        df_well = df_IPC[df_IPC['Well']==well].iloc[0]
        #
        well_ql = df_well['Liquid_rate']
        well_fw = df_well['Water_cut']
        well_GOR = df_well['GOR']
        well_RMU = df_well['RMU']
        well_OE = df_well['Uptime']
        try:
            rmu_GOR = d_RMU_MBAL[well_RMU][i_RMUMBAL_GOR]
        except:
            rmu_GOR = 342.0
        #
        # convert WC to fractional value
        well_fw *= 0.01
        #
        # build list of well IPC values
        f_well_qo = well_ql * (1.0 - well_fw) * f_bo * 1000.0                    # rb/d oil (note IPC/IIC are in mstb/d)
        f_well_qw = well_ql * well_fw * f_bw * 1000.0                            # rb/d water (note IPC/IIC are in mstb/d)
        f_well_qg = (well_ql * (1.0 - well_fw))*(max(0,well_GOR - rmu_GOR))*f_bg # rb/d gas
        #
        f_well_offtake = -1.0*(f_well_qo + f_well_qw + f_well_qg) # total rb/d offtake (including free gas phase) - negative = offtake convention
        #
        # append values to list, also multiply through by OE of the well
        l_df_well_data.append([well,well_RMU,-1.0*f_well_qo*well_OE,-1.0*f_well_qw*well_OE,-1.0*f_well_qg*well_OE,f_well_offtake*well_OE])
        #
    #
    # do the same for the injectors
    for well in df_IIC['Well'].unique():
        #
        # slice df and get information
        df_well = df_IIC[df_IIC['Well']==well].iloc[0]
        #
        well_qw = df_well['Injection_rate']
        well_OE = df_well['Uptime']
        well_RMU = df_well['RMU']
        #
        # derive injection rate
        f_well_qw = well_qw * f_bw * 1000.0                                      # rb/d water (note IPC/IIC are in mstb/d)
        #
        f_well_offtake = f_well_qw # total rb/d intake, positive = intake convention
        #
        # append values to list, also multiply through by OE of the well
        l_df_well_data.append([well,well_RMU,0.0,f_well_qw*well_OE,0.0,f_well_offtake*well_OE])
    #
    # build df from the collated list 
    df_offtake = pd.DataFrame(l_df_well_data,columns=['Well','RMU','qo_rbpd','qw_rbpd','qg_rbpd','offtake_rate_rbpd'])
    #
    # now loop through the df and compute RMU level net offtake
    l_rmu_voidage = []
    for rmu in Q204_RMU_dictionary:
        #
        # slice df on wells in the rmu
        df_rmu = df_offtake[df_offtake['Well'].isin(Q204_RMU_dictionary[rmu][i_dict_rmuwell_wells])]
        #
        # sum the offtakes
        rmu_position = df_rmu['offtake_rate_rbpd'].sum()
        #
        # store in list
        l_rmu_voidage.append([rmu,rmu_position])
    #
    # create df of rmu positions
    df_rmu_voidage = pd.DataFrame(l_rmu_voidage,columns=['RMU','voidage_rbpd'])
    #
    return df_rmu_voidage

In [ ]:
def rmu_forecast(forecast_RMU_rollup, historical_pressures, PRES_data_all):
    #
    # Code transform to pull most recent reservoir pressure and forecast based on this
    # (to do: stop HM / start forecast at set point)
    #
    # if you get errors because of positional locators, check the historical data RMU rollup and compressibility history match
    #   definition of l_rmu: sometimes I set that to a single RMU to speed up the runs
    #   (but it means this doesn't work! - you should put in an error check in the code, fun friday job)
    #
    # take copies of input dfs
    df_voidage = forecast_RMU_rollup.copy(deep=True)
    df_historical_pressure = historical_pressures.copy(deep=True)
    df_PRES = PRES_data_all.copy(deep=True)
    #
    i_use_global_compressibility = True # if true, pull GOR/aquifer/compressibility data from global dictionary d_RMU_MBAL, otherwise pull from input table
    #
    # set up RMU MBAL dictionary
    if i_use_global_compressibility == True:
        # use the global dictionary
        d_RMU_match = d_RMU_MBAL
        #
    else:
        # use the input df
        df_RMU_MBAL = Forecasting_inputs_RMU_compressibility.copy(deep=True)
        #
        # sort the df to be in the same expected order as the global dictionary
        df_RMU_MBAL = df_RMU_MBAL[['RMU','Solution_GOR','Aquifer','Aquifer_delay','Compressibility','Initial_datum_pressure','Initial_datum_date']]
        df_RMU_MBAL.set_index('RMU',inplace=True,drop=True)
        #
        # convert df into dictionary (I don't know why Pandas doesn't do the obvious key:list from columns, so use dictionary logic instead)
        d_RMU_match = {df_RMU_MBAL.index.values[i]:df_RMU_MBAL.values.tolist()[i] for i in range(len(df_RMU_MBAL.index.values))}
    #
    # debugger
    l_rmus = rmu_list
    # loop over RMUs and make forecast
    #
    for rmu in l_rmus:
        #
        # get the voidage for the rmu
        f_rmu_voidageposition = df_voidage[df_voidage['RMU']==rmu]['voidage_rbpd'].iloc[0]
        #
        # get the compressibility HM pressure
        df_rmu_hm_pres = df_historical_pressure[df_historical_pressure['rmu']==rmu].reset_index(drop=True)
        #
        # check if forecast start time has pressure, otherwise reset to last HM point
        my_forecast_start = pd.to_datetime(dt_forecast_start,utc=True)
        #
        if my_forecast_start not in df_rmu_hm_pres['time']:
            my_forecast_start = df_rmu_hm_pres['time'].iloc[-1]
            print(str('Warning: RMU '+str(rmu)+' does not have predicted pressure at wanted time'))
            print(str('Prediction start point reset from '+str(dt_forecast_start)+' to '+str(my_forecast_start)))
        #
        # get initial pressure from estimated PRES from model
        df_rmu_estimate = df_rmu_hm_pres[df_rmu_hm_pres['time']==my_forecast_start]
        pinit_est = df_rmu_estimate['estimated_PRES'].iloc[0]
        #
        # also get from average PRES within 4 weeks
        dt_start_estimate = my_forecast_start - pd.Timedelta(28,'d')
        #
        # slice PRES ALL data frame on producers in RMU
        l_rmu_producers = []
        #
        for well in Q204_RMU_dictionary[rmu][i_dict_rmuwell_wells]:
            if well[1] == 'P':
                l_rmu_producers.append(well)
        #
        df_RMU_PRES = df_PRES[df_PRES['Well'].isin(l_rmu_producers)]
        #
        # guess initial pressure from PBUs
        l_PRES = df_RMU_PRES[df_RMU_PRES['time']>dt_start_estimate]['Datum_PRES_psi_corrected'].tolist()
        if l_PRES > 0:
            pinit_pres = np.nanmean(l_PRES)
        else:
            pinit_pres = pinit_est
        #
        # also get the starting pressure (for aquifer calcs), compressibility and aquifer strength/delay
        f_rmu_pi = d_RMU_match[rmu][i_RMUMBAL_pinit]
        f_rmu_c  = d_RMU_match[rmu][i_RMUMBAL_compressibility]
        f_rmu_aq = d_RMU_match[rmu][i_RMUMBAL_aquifer]
        i_rmu_aq_delay = d_RMU_match[rmu][i_RMUMBAL_aquifer_delay]
        #
        # get the historical aquifer data
        #l_aqdata_historical = df_rmu_hm_pres['cumulative_aquifer'].tolist()
        l_p_hm = df_rmu_hm_pres['estimated_PRES'].tolist()
        i_aqdata_len = len(l_p_hm) # fix this so we can effectively merge the aquifer forecast with history
        #
        # loop over the times and compute estimated pressures
        l_rmu_df = []
        l_rmu_dates = [my_forecast_start]
        l_rmu_p_est = [pinit_est]        # initialise list of predicted pressures
        l_rmu_p_pres = [pinit_pres]        # initialise list of predicted pressures
        l_rmu_cumvoidage = [df_rmu_estimate['cumulative_voidage'].iloc[0]]    # initialise cumulative voidage
        l_rmu_cumaquifer = [df_rmu_estimate['cumulative_aquifer'].iloc[0]]    # initialise cumulative aquifer influx
        #
        for i in range(1,31): # don't start from zero as we will call previous time points and python [-1] list won't do what you initially think
            #
            # get the instantaneous voidage offtake
            my_offtake = f_rmu_voidageposition*1e-6 # note offtake rates need to be converted to mmrbd
            #
            # get aquifer influx (q_in (rb/d) = PI*dP, assume paq ~ pinit), use historical data set for this NOT the current (ongoing) data set
            if (i_aqdata_len+(i-1)) > i_rmu_aq_delay: # make sure we also include historical data for the delay computation to avoid unintentional seam
                my_aquifer = f_rmu_aq * (f_rmu_pi - l_p_hm[(i_aqdata_len + (i-1)) - i_rmu_aq_delay - 1]) # lower pres = positive aquifer influx
            else:
                my_aquifer = 0.0
            #
            # convert aquifer influx into mmbbl/d
            my_aquifer *= 1e-6 # note offtake rates are in mmrbpd
            #
            # update pressure estimate (note convention: negative = offtake, positive = intake)
            l_rmu_p_est.append(l_rmu_p_est[i-1] + f_rmu_c * (my_offtake + my_aquifer))
            l_rmu_p_pres.append(l_rmu_p_pres[i-1] + f_rmu_c * (my_offtake + my_aquifer))
            #
            # append cum voidage values (for plotting) and date of forecast
            l_rmu_cumvoidage.append(l_rmu_cumvoidage[i-1] + my_offtake)
            l_rmu_cumaquifer.append(l_rmu_cumaquifer[i-1] + my_aquifer)
            l_p_hm.append(l_rmu_p_est[i]) # need to add pressures to the history list for short-acting (less than prediction scale) aquifers
            l_rmu_dates.append(l_rmu_dates[i-1] + pd.Timedelta(1,'d'))
            #
            # append data to df list
            l_rmu_df.append([l_rmu_dates[i],rmu,l_rmu_p_est[i],l_rmu_p_pres[i],l_rmu_cumvoidage[i],l_rmu_cumaquifer[i]])
            #
            # debugging
            # if rmu == 'S1SE':
            #    print(rmu,i,l_rmu_p_est[i-1],l_rmu_p_est[i],my_offtake,my_aquifer)
                # if i == 1:
                #    print(rmu,i,my_aquifer,l_rmu_dates[i],l_p_hm[(i_aqdata_len + (i-1)) - i_rmu_aq_delay - 1],df_rmu_hm_pres['time'].iloc[(i_aqdata_len + (i-1)) - i_rmu_aq_delay - 1])
                # else:
                #    print(rmu,i,my_aquifer,l_rmu_dates[i],l_p_hm[(i_aqdata_len + (i-1)) - i_rmu_aq_delay - 1])
        #
        # add data to fresh RMU df
        df_rmu_forecast = pd.DataFrame(l_rmu_df,columns=['time','rmu','forecast_P_est','forecast_P_pres','cumulative_voidage','cumulative_aquifer'])
        #
        # concatenate RMU roundups
        try:
            df_rmu_forecast_all = pd.concat([df_rmu_forecast_all,df_rmu_forecast],ignore_index=True)
        except:
            df_rmu_forecast_all = df_rmu_forecast.copy(deep=True)
        #
    #
    return df_rmu_forecast_all
            

In [ ]:
def compressibility_plotter(historical_pressures, PRES_data_all, rmu_forecast):
    #
    # eventually move this into a large plotter to show ALL regions on one matplotlib plot
    #
    # options
    plot_RMU = 'RMU1'
    i_plot_pred = True
    #
    # grab the dfs and slice on the segment only
    df_hm_pres = historical_pressures.copy(deep=True)
    df_hm_pres = df_hm_pres[df_hm_pres['rmu']==plot_RMU].sort_values(by='time')
    df_pred_pres = rmu_forecast.copy(deep=True)
    df_pred_pres = df_pred_pres[df_pred_pres['rmu']==plot_RMU].sort_values(by='time')
    #
    # build list of wells (producers)
    my_plot_well_list = []
    for well in Q204_RMU_dictionary[plot_RMU][i_dict_rmuwell_wells]:
        if well[1] == 'P':
            my_plot_well_list.append(well)
    #
    df_history_pres = PRES_data_all.copy(deep=True)
    df_history_pres = df_history_pres[df_history_pres['Well'].isin(my_plot_well_list)]
    #
    # set up matplotlib gridspaces
    plt.subplots(figsize=(15,10))
    gs = gridspec.GridSpec(2, 1, height_ratios=[4, 1])
    gs.update(hspace=0.1)
    #
    ax = plt.subplot(gs[0])
    #
    # plot the historical data
    for well in df_history_pres['Well'].unique():
        df_plot  = df_history_pres[df_history_pres['Well']==well]
        ax.scatter(df_plot['time'],df_plot['Datum_PRES_psi_corrected'],label=well)
    #
    # plot the history match pressures
    ax.plot(df_hm_pres['time'],df_hm_pres['estimated_PRES'],c='black',linestyle='solid',label='History match')
    plt.ylabel('Datum pressure (psia)')
    #
    # if requested, plot the predictions
    if i_plot_pred == True:
        ax.plot(df_pred_pres['time'],df_pred_pres['forecast_P_est'],c='black',linestyle='dashed',label='Prediction (from HM estimate)')
        ax.plot(df_pred_pres['time'],df_pred_pres['forecast_P_pres'],c='green',linestyle='dashed',label='Prediction (from PRES average)')
    #
    #
    plt.legend()
    #
    ax2 = plt.subplot(gs[1], sharex = ax)
    ax2.plot(df_hm_pres['time'],df_hm_pres['cumulative_voidage'],c='black',linestyle='solid',label='Cumulative voidage')
    ax2.plot(df_hm_pres['time'],df_hm_pres['cumulative_aquifer'],c='blue',linestyle='solid',label='Cumulative aquifer influx')
    #
    if i_plot_pred == True:
        ax2.plot(df_pred_pres['time'],df_pred_pres['cumulative_voidage'],c='black',linestyle='dashed',label='_nolabel_')
        ax2.plot(df_pred_pres['time'],df_pred_pres['cumulative_aquifer'],c='blue',linestyle='dashed',label='_nolabel_ aquifer influx')
    #
    plt.ylabel('Net voidage (mmrb)')
    #
    plt.legend()
    #
    plt.tight_layout()
    plt.show()
        

In [ ]:
def RMU_allocation_split(RMU_rollup_historical, Rolled_well_data):
    #
    # use this Code Workbook to split out allocation from the various producers/injectors in each segment
    # (helpful for history matching)
    #
    # note that for now this only plots the primary wells contributing, add other wells with the fudge below
    #
    plot_RMU = 'RMU1'
    plot_wells = ['P01']
    #
    # copy dfs and slice on the RMU
    df_RMU = RMU_rollup_historical.copy(deep=True)
    df_RMU = df_RMU[df_RMU['rmu']==plot_RMU]
    #
    l_wells = Q204_RMU_dictionary[plot_RMU][i_dict_rmuwell_wells] # build list of wells in RMU from global list
    l_wells = l_wells + plot_wells
    df_well = Rolled_well_data.copy(deep=True)
    df_well = df_well[df_well['well'].isin(l_wells)]
    #
    # plot the data
    plt.subplots(figsize=(15,10))
    #
    # RMU allocation
    plt.plot(df_RMU['time'],df_RMU['cum_offtake'],c='black',label='RMU')
    #
    # well allocation
    for well in df_well['well'].unique():
        df_plot_well = df_well[df_well['well']==well]
        plt.plot(df_plot_well['datetime'],df_plot_well['cum_offtake'],label=well)
    #
    # plot the legend
    plt.legend()
    plt.ylabel('Cumulative intake/offtake (mmrb)')
    #
    # show the plot
    plt.show()